In [ ]:
#%load_ext autoreload
#%autoreload 2

import pandas as pd
import numpy as np

import hvplot.pandas
import holoviews as hv
import panel as pn

In [ ]:
import openpyxl

In [ ]:
df=pd.read_excel('./schout_2_Georgiana cross section.xlsx')


## Average over time the velocities in x,y,z 
These are the coarse values that will become the input to the ANN

Each input for a time stamp has 3 values of average x,y,z velocities. 

These will be mapped to 13 (xy) into 9 (z) values, i.e. 117 value array for each time step

Furthermore a phase variable based on a cycle of 24 hrs, 15min, i.e. tidal signal will be added to the above inputs for a total of 4 inputs per time (each timestamp will become one training sample)

The value of the phase variable can vary between 0 and 1 with a period of 24 hours and 15 mins

In [ ]:
df_input=df.groupby('time').agg('mean')[['xVel','yVel','zVel']]
df_input

In [ ]:
df_input['phase']=np.arange(0,1,1/100.)[:96]

In [ ]:
df_input

In [ ]:
df_output_depth_averaged=df.groupby(['time','x','y']).mean()

In [ ]:
dfg_output=df.groupby(['time'])
arr1=np.empty((96,117,3))
for i,(n,g) in enumerate(dfg_output):
    arr1[i]=g[['xVel','yVel','zVel']].values

## Output has shape 117 x 3(velocities x,y,z)
There are 96 time samples

Y is the target or output vector

In [ ]:
arr1.shape

In [ ]:
Y=arr1

## Input has shape of 4 values
There are 96 samples (time steps)

X is the input vector or labels

In [ ]:
df_input.values.shape

In [ ]:
X=df_input.values

## Design a neural network 

The neural network should take 3 velocities and phase signal ( 25 hour approx) and output an array of 117 (xy (13) and z (9)) x 3 velocities

In [ ]:
# tensorflow with keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
# sci-kit learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

## Scale the X and Y values to normalize. 

This may not be needed here but generally a good practice in numerics 

In [ ]:
xscaler=MinMaxScaler()
xx=xscaler.fit_transform(X)
xx.shape

In [ ]:
yyt=np.empty(Y.shape)

yscaler_array = [MinMaxScaler() for i in range(Y.shape[2])]
for i in range(Y.shape[2]):
    yyt[:,:,i] = yscaler_array[i].fit_transform(Y[:, :, i])

In [ ]:
yyt.shape

In [ ]:
model = keras.Sequential(
    [
        layers.Input(4),  # time x # of inputs
        layers.Dense(117*10, activation='relu'),
        layers.Dense(117*5, activation='relu'),
        layers.Dense(117*3, activation='relu'),
        layers.Reshape((117,3))
        #layers.Conv1D(filters=10, kernel_size=10, activation='relu'),
        #layers.Conv1D(filters=5, kernel_size=5, activation='relu'),
        # layers.LSTM(20,activation='relu'),
        #layers.Dense(10, activation="relu"),
        #layers.Dense(1, activation="relu")
    ])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

In [ ]:
model.output_shape

In [ ]:
yyt.shape

In [ ]:
history = model.fit(
    xx,
    yyt,
    epochs=500,
    batch_size=128,
    validation_split=0.1,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, mode="min")
    ],
)


In [ ]:
yyp=model.predict(xx)

In [ ]:
yyp.shape,yyt.shape

In [ ]:
hv.Curve(history.history['loss'],label='loss')

## Show comparison of predictions vs values

In [ ]:
def compare(index=0,vel_dim=0):
    return hv.Curve(yyp[index,:,vel_dim],label='predicted')*hv.Curve(yyt[index,:,vel_dim],label='target')

In [ ]:
hv.DynamicMap(compare,kdims=['index','vel_dim']).redim.range(index=(0,96),vel_dim=(0,3))